In [1]:
import pandas as pd
from datetime import datetime

session = "181"
pos_dir = './out/pos'
data_dir = "./data/{}".format(session)


In [2]:
"""
@author Riordan Dervin Alfredo <riordan.alfredo@monash.edu>
"""
# Run to extract all pozyx position data
# json -> csv

data_in = "{}/{}.json".format(data_dir, session)
sync_path = "{}/sync.txt".format(data_dir)
%run pozyx_extraction.py -p $data_in -o "out/pos/" -s {sync_path}


all done!


In [21]:
colour = "YELLOW"
file_date = "18-Aug-2021_15-33-14-715"

In [22]:
# install it separately https://ffmpeg.org/. Use this to re-sample audio data to 48KHz
# 
audio_in = "{}/simulation_{}_{}_audio.wav".format(data_dir,colour,file_date)
audio_out = "{}/sim_{}.wav".format(data_dir,colour)

print(audio_in)
print(audio_out)

!ffmpeg -i {audio_in} -ar 48000 {audio_out}

./data/181/simulation_YELLOW_18-Aug-2021_15-33-14-715_audio.wav
./data/181/sim_YELLOW.wav


ffmpeg version 2022-07-04-git-dba7376d59-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12.1.0 (Rev2, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-liblensfun --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --e

In [23]:
# TODO: replace audio file name!
# A script to convert speech to text, wav -> csv
audio_sim_dir = './out/audio-sim'
audio_csv_out = "{}/{}_{}.csv".format(audio_sim_dir, session, colour)

%run hive_automation.py -a {audio_out} -o {audio_csv_out} -s "145" -w "1" -t "3"

start
here
 before vad
1646.0509791666666
thread_started: speech2text thread 0


  0%|          | 0/24 [00:00<?, ?it/s]

thread_started: speech2text thread 1
thread_started: speech2text thread 2


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

thread_finished: speech2text thread 0
thread_finished: speech2text thread 2
thread_finished: speech2text thread 1
object


In [18]:
# combine position and audio information

dfp = pd.read_csv('{}/{}_{}.csv'.format(pos_dir,session,colour))
dfa = pd.read_csv('{}/{}_{}.csv'.format(audio_sim_dir,session,colour))

dfp.head()

,Unnamed: 0,audio_start_timestamp,audio time,x,y
0,0,1.629265e+09,0:11:45,1506.267849,-1330.691270
1,1,1.629265e+09,0:11:46,1571.551551,-1209.885914
2,2,1.629265e+09,0:11:47,1711.205829,-966.520268
3,3,1.629265e+09,0:11:48,1809.562382,-785.917458
4,4,1.629265e+09,0:11:49,1841.747898,-669.764675


In [19]:
res = pd.merge(dfp, dfa, on="audio time")
final = res.drop(labels=["Unnamed: 0_x", "Unnamed: 0_y", "session"], axis=1)
final['tagId'] = colour
final.head()

,audio_start_timestamp,audio time,x,y,audio,tagId
0,1.629265e+09,0:11:45,1506.267849,-1330.691270,0,GREEN
1,1.629265e+09,0:11:46,1571.551551,-1209.885914,0,GREEN
2,1.629265e+09,0:11:47,1711.205829,-966.520268,0,GREEN
3,1.629265e+09,0:11:48,1809.562382,-785.917458,0,GREEN
4,1.629265e+09,0:11:49,1841.747898,-669.764675,0,GREEN


In [20]:

result_dir =  "./out/result"
result_csv = "{}/{}_{}.csv".format(result_dir,session,colour)
final.to_csv(result_csv, sep=',', encoding='utf-8', index=False)

# Save to front-end too!
# final.to_csv("{}/{}_{}.csv".format(client_dir,session,colour), sep=',', encoding='utf-8', index=False)

# Before you continue...
Please repeat all available colours before combining them in the following code cells

In [ ]:

# combine all data
df = pd.concat(map(pd.read_csv, [
    '{}/{}_RED.csv'.format(result_dir, session), 
    '{}/{}_YELLOW.csv'.format(result_dir, session), 
    '{}/{}_BLUE.csv'.format(result_dir, session),
    '{}/{}_GREEN.csv'.format(result_dir, session)]), ignore_index=True)

df = df.sort_values(by='audio time')

In [ ]:
client_dir = "../client/src/projects/hive/data"
df.to_csv("{}/{}_all.csv".format(client_dir,session), sep=',', encoding='utf-8', index=False)

df